# Import Libraries

In [1]:
import pandas as pd

from pathlib import Path

# Data Acquisition

In [2]:
data_path = Path("./Raw data/")

dfs = []

for path in data_path.iterdir():
  if path.suffix == ".csv":
    dfs.append(pd.read_csv(path, thousands=','))

In [3]:
for df in dfs:
  display(df.head(1))

,kabupaten,tahun,Indeks Kemahalan Konstruksi Kabupaten/Kota
0,Simeulue,2020,109.99


,kabupaten,tahun,IPM
0,Simeulue,2020,66.03


,kabupaten,tahun,PDRB
0,Simeulue,2020,2274.36


,Kabupaten,Tahun,Persentase Penduduk Miskin (P0)
0,Simeulue,2020,18.49


,Kabupaten,Tahun,Prevalensi Ketidakcukupan Konsumsi Pangan (Persen) Per Kabupaten_kota
0,Simeulue,2020,11.44


,Kabupaten,Tahun,Proporsi Perempuan Pernah Kawin 15-49 tahun yang Melahirkan Anak Lahir Hidup Yang Pertama Kali Berumur Kurang dari 20 tahun Menurut Kabupaten/Kota
0,Simeulue,2020,0.291


# Checking Data Integrity

Normalisasi nama kolom:
- Mengubah semua huruf jadi lowercase
- Mengubah spasi menjadi underscore "_"
- Menghapus whitespace pada awal dan akhir nama kolom

In [4]:
for i, df in enumerate(dfs):
  dfs[i] = df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"))

Drop baris duplikat (Kesalahan saat input data)

In [5]:
for i, df in enumerate(dfs):
  dfs[i] = df.drop_duplicates()

Uji banyak observasi pada tiap dataframe

In [6]:
for i, df in enumerate(dfs):
  print(f"Dataset ke-{i} ({df.columns[2]}) shape: {df.shape}")

Dataset ke-0 (indeks_kemahalan_konstruksi_kabupaten/kota) shape: (1542, 3)
Dataset ke-1 (ipm) shape: (1542, 3)
Dataset ke-2 (pdrb) shape: (1542, 3)
Dataset ke-3 (persentase_penduduk_miskin_(p0)) shape: (1542, 3)
Dataset ke-4 (prevalensi_ketidakcukupan_konsumsi_pangan_(persen)_per_kabupaten_kota) shape: (1542, 3)
Dataset ke-5 (proporsi_perempuan_pernah_kawin_15-49_tahun_yang_melahirkan_anak_lahir_hidup_yang_pertama_kali_berumur_kurang_dari_20_tahun_menurut_kabupaten/kota) shape: (1542, 3)


# Data Merging

In [7]:
df_merged = None

for df in dfs:
  if df_merged is None:
    df_merged = df
  else:
    df_merged = pd.merge(df_merged, df, on=["kabupaten", "tahun"], how="inner")

df = df_merged

In [8]:
display(df.head(3))

df.info()

,kabupaten,tahun,indeks_kemahalan_konstruksi_kabupaten/kota,ipm,pdrb,persentase_penduduk_miskin_(p0),prevalensi_ketidakcukupan_konsumsi_pangan_(persen)_per_kabupaten_kota,proporsi_perempuan_pernah_kawin_15-49_tahun_yang_melahirkan_anak_lahir_hidup_yang_pertama_kali_berumur_kurang_dari_20_tahun_menurut_kabupaten/kota
0,Simeulue,2020,109.99,66.03,2274.36,18.49,11.44,0.291
1,Aceh Singkil,2020,99.72,68.94,2422.61,20.20,7.19,0.283
2,Aceh Selatan,2020,98.07,67.12,5530.75,12.87,8.57,0.287


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1542 entries, 0 to 1541
Data columns (total 8 columns):
 #   Column                                                                                                                                              Non-Null Count  Dtype  
---  ------                                                                                                                                              --------------  -----  
 0   kabupaten                                                                                                                                           1542 non-null   object 
 1   tahun                                                                                                                                               1542 non-null   int64  
 2   indeks_kemahalan_konstruksi_kabupaten/kota                                                                                                          1542 non-null   float64
 3   ipm 

Uji kesamaan format nama kabupaten

In [9]:
for idx, other in enumerate(dfs):
  comparator_df = df_merged.copy()
  missing_mask = ~other["kabupaten"].isin(comparator_df["kabupaten"])
  missing = other.loc[missing_mask, "kabupaten"].unique()

  print(f"dfs[{idx}] format berbeda: {len(missing)} -> {missing}\n")

dfs[0] format berbeda: 0 -> []

dfs[1] format berbeda: 0 -> []

dfs[2] format berbeda: 0 -> []

dfs[3] format berbeda: 0 -> []

dfs[4] format berbeda: 0 -> []

dfs[5] format berbeda: 0 -> []



# Data Renaming

In [10]:
df = df.rename(columns={
    "ipm": "indeks_pembangunan_manusia",
    "indeks_kemahalan_konstruksi_kabupaten/kota": "indeks_kemahalan_konstruksi",
    "pdrb": "produk_domestik_regional_bruto",
    "persentase_penduduk_miskin_(p0)": "persentase_penduduk_miskin",
    "prevalensi_ketidakcukupan_konsumsi_pangan_(persen)_per_kabupaten_kota": "prevalensi_ketidakcukupan_konsumsi_pangan",
    "proporsi_perempuan_pernah_kawin_15-49_tahun_yang_melahirkan_anak_lahir_hidup_yang_pertama_kali_berumur_kurang_dari_20_tahun_menurut_kabupaten/kota": "proporsi_perempuan_melahirkan_anak_pertama_kurang_dari_20_tahun",
})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1542 entries, 0 to 1541
Data columns (total 8 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   kabupaten                                                        1542 non-null   object 
 1   tahun                                                            1542 non-null   int64  
 2   indeks_kemahalan_konstruksi                                      1542 non-null   float64
 3   indeks_pembangunan_manusia                                       1542 non-null   float64
 4   produk_domestik_regional_bruto                                   1542 non-null   float64
 5   persentase_penduduk_miskin                                       1542 non-null   float64
 6   prevalensi_ketidakcukupan_konsumsi_pangan                        1542 non-null   float64
 7   proporsi_perempuan_melahirkan_anak_pertama_

# Export Data

In [11]:
df.to_csv("./Processed data/processed_data.csv", index=False)